In [54]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.metrics import accuracy_score, r2_score
import numpy as np
import random
import pandas as pd
from tqdm import tqdm
from lime.lime_tabular import LimeTabularExplainer
import numpy as np
from sklearn.datasets import fetch_openml
import pandas as pd
from sklearn.metrics import jaccard_score
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
import pickle
from joblib import load 
from scipy.optimize import minimize

from sklearn.preprocessing import StandardScaler, LabelEncoder

In [55]:
import pandas as pd
import random
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.metrics import accuracy_score, r2_score
from lime.lime_tabular import LimeTabularExplainer
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import Lasso
from sklearn.datasets import fetch_openml
from joblib import load
from tqdm import tqdm
import warnings

In [56]:
def preprocess_data(df, target_column, categorical_columns=[], n_train=None):
    # Remove rows where target contains NaN values
    df = df.dropna(subset=[target_column])
    
    # Frequency encoding for categorical features
    for col in categorical_columns:
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col])

    # Split the dataset into features (X) and target (y)
    X = df.drop(target_column, axis=1)  # Features
    y = df[target_column]  # Target

    # Get number of features (p) and total samples (n_total)
    p = X.shape[1]
    n_total = X.shape[0]

    # Train-test split
    if n_train:
        X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=n_train, random_state=42)
    else:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Standardize the features
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    return X_train, X_test, y_train, y_test, n_total, p, X.columns.tolist()  # Return feature names

parkinsons_df = pd.read_csv('data/parkinsons.data')
parkinsons_df = parkinsons_df.drop('name', axis=1)
target_column = 'status'
categorical_columns = []
X_train_parkinsons, X_test_parkinsons, y_train_parkinsons, y_test_parkinsons, n_total_parkinsons, p_parkinsons, feature_names_parkinsons = preprocess_data(parkinsons_df, 'status', [], n_train=175)

cancer_df = pd.read_csv('data/cancer.csv')
cancer_df = cancer_df.drop(['id', 'Unnamed: 32'], axis=1)
cancer_df['diagnosis'] = cancer_df['diagnosis'].map({'M': 1, 'B': 0})
target_column = 'diagnosis'
# Updated unpacking for the Cancer dataset
X_train_cancer, X_test_cancer, y_train_cancer, y_test_cancer, n_total_cancer, p_cancer, feature_names_cancer = preprocess_data(cancer_df, 'diagnosis', [], n_train=512)


adult_df = pd.read_csv('data/adult.csv')
adult_df = adult_df.replace('?', np.nan)  # Handle missing values
adult_df = adult_df.dropna()  # Drop any rows with missing values
categorical_columns = ['workclass', 'education', 'marital.status', 'occupation', 'relationship', 'race', 'sex', 'native.country']
target_column = 'income'
adult_df[target_column] = adult_df[target_column].map({'<=50K': 0, '>50K': 1})
X_train_adult, X_test_adult, y_train_adult, y_test_adult, n_total_adult, p_adult, feature_names_adult = preprocess_data(adult_df, target_column, categorical_columns)
boston = fetch_openml(name="boston", version=1, as_frame=True)
X, y = boston.data, boston.target
boston_df = X.copy()
boston_df['MEDV'] = y  # MEDV is the house price (target)
target_column = 'MEDV'
categorical_columns = []
X_train_boston, X_test_boston, y_train_boston, y_test_boston, n_total_boston, p_boston, feature_names_boston = preprocess_data(boston_df, 'MEDV', [], n_train=455)
bodyfat_df = pd.read_csv('data/bodyfat.csv')
target_column = 'BodyFat'
X_train_bodyfat, X_test_bodyfat, y_train_bodyfat, y_test_bodyfat, n_total_bodyfat, p_bodyfat, feature_names_bodyfat = preprocess_data(bodyfat_df, 'BodyFat', [], n_train=226)


In [57]:
def calculate_jaccard_distance(X_i, X_j):
    """
    Calculate Jaccard distance between two sets of features.
    X_i and X_j are expected to be lists of top features.
    """
    set_i = set(X_i)
    set_j = set(X_j)
    
    intersection = len(set_i.intersection(set_j))
    union = len(set_i.union(set_j))
    
    if union == 0:
        return 0.0  # If both sets are empty
    return 1 - (intersection / union)  # Jaccard distance


In [58]:
def stability_score(D_final, n_samples=10, n_iterations=50):
    """
    Calculate stability score over n_samples and n_iterations.
    D_final is the surrogate data returned by unravel.
    """
    all_explanations = []
    
    for _ in range(n_iterations):
        current_explanations = []
        for sample in random.sample(D_final, n_samples):
            # Assume that we have a method to get top-5 features based on the model predictions
            top_features = get_top_features(sample[0])  # Replace with actual feature extraction logic
            current_explanations.append(top_features)
        
        all_explanations.append(current_explanations)

    # Calculate Jaccard distance for all combinations
    jaccard_distances = []
    
    for i in range(n_iterations):
        for j in range(i + 1, n_iterations):
            for k in range(n_samples):
                jaccard_distances.append(calculate_jaccard_distance(all_explanations[i][k], all_explanations[j][k]))

    # Average the Jaccard distances
    avg_stability = np.mean(jaccard_distances)
    return avg_stability

def get_top_features(instance):
    """
    Mock function to extract top-5 features for the instance.
    You should replace this with your actual feature importance logic.
    """
    # For demonstration purposes, we randomly select 5 features
    return random.sample(range(1, 15), 5)  # Adjust according to actual feature space


In [59]:
def init_gpr():
    # Initialize the Gaussian Process Regressor with an ARD kernel
    kernel = C(1.0, (1e-3, 1e3)) * RBF(length_scale=10.0, length_scale_bounds=(1e-2, 1e5))
    return GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10)

def acquisition_function(X_sample, model, epsilon=1e-6):
    # Compute acquisition function (Expected Improvement)
    mean, std = model.predict(X_sample, return_std=True)
    return mean + std + epsilon  # Add epsilon to avoid zero improvement

In [60]:
def optimize_acquisition_function(model, exploration_domain, n_samples=100):
    # Randomly sample from the exploration domain and optimize acquisition function
    # exploration_domain is now just (lower_bound, upper_bound)
    lower_bound, upper_bound = exploration_domain
    
    # Generate random samples based on the dimensionality of x0
    dimension = len(lower_bound)  # Assuming lower_bound is an array-like structure
    X_random = np.random.uniform(lower_bound, upper_bound, (n_samples, dimension))
    
    scores = acquisition_function(X_random, model)
    best_index = np.argmax(scores)
    return X_random[best_index]

In [61]:
def unravel(model_path, x0, n_iterations=10):
    # Load the pre-trained model
    model = load(model_path)
    
    # Initialize data set D
    D = [(x0, model.predict([x0])[0])]  # Initial data point with prediction

    # Set exploration domain
    sigma_D = np.std([data[1] for data in D])  # Calculate standard deviation from current predictions
    exploration_domain = (x0 - sigma_D, x0 + sigma_D)  # Adjusted exploration domain

    # Initialize GPR
    gpr = init_gpr()

    for l in range(n_iterations):
        # Prepare data for GPR
        X_train, y_train = zip(*D)
        gpr.fit(np.array(X_train), np.array(y_train))

        # Optimize acquisition function to find the next sample
        x_next = optimize_acquisition_function(gpr, exploration_domain, n_samples=100)
        
        # Obtain new prediction and add to dataset
        y_next = model.predict([x_next])[0]
        D.append((x_next, y_next))

        # Update exploration domain
        exploration_domain = (x_next - sigma_D, x_next + sigma_D)

    return D  # Return the final surrogate data

In [62]:
# Define parameters for each dataset/model
datasets = {
    'Parkinsons': {
        'model_path': 'models/parkinsons_svm_model.pkl',
        'initial_instance': np.random.rand(22),  # Adjust based on feature number
        'iterations': 10
    },
    'Cancer': {
        'model_path': 'models/cancer_svm_model.pkl',
        'initial_instance': np.random.rand(30),  # Adjust based on feature number
        'iterations': 10
    },
    'Adult': {
        'model_path': 'models/adult_svm_model.pkl',
        'initial_instance': np.random.rand(14),  # Adjust based on feature number
        'iterations': 10
    },
    'Boston': {
        'model_path': 'models/boston_extra_trees_model.pkl',
        'initial_instance': np.random.rand(13),  # Adjust based on feature number
        'iterations': 10
    },
    'BodyFat': {
        'model_path': 'models/bodyfat_extra_trees_model.pkl',
        'initial_instance': np.random.rand(14),  # Adjust based on feature number
        'iterations': 10
    }
}


In [63]:
stability_scores = {
    'Dataset': [],
    'Stability Score': []
}


In [64]:
# Run UnRAvEL for each dataset and collect stability scores
for dataset, params in datasets.items():
    D_final = unravel(params['model_path'], params['initial_instance'], n_iterations=params['iterations'])
    
    # Assuming some method to compute stability score here, you might have to implement this
    stability_score = np.random.rand()  # Placeholder for the actual stability score computation
    stability_scores['Dataset'].append(dataset)
    stability_scores['Stability Score'].append(stability_score)


c:\Users\ADITI ROY\Desktop\BML_PROJECT\.venv\Lib\site-packages\sklearn\gaussian_process\_gpr.py:659: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
c:\Users\ADITI ROY\Desktop\BML_PROJECT\.venv\Lib\site-packages\sklearn\gaussian_process\_gpr.py:659: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
c:\Users\ADITI ROY\Desktop\BML_PROJECT\.venv\Lib\site-packages\sklearn\gaussian_process\_gpr.py:659: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data 

In [11]:
stability_df = pd.DataFrame(stability_scores)


In [12]:
print(stability_df)


      Dataset  Stability Score
0  Parkinsons         0.296821
1      Cancer         0.706339
2       Adult         0.052505
3      Boston         0.857602
4     BodyFat         0.486845


In [13]:
stability_df_LIME=pd.read_csv('stability_results_LIME.csv')

In [14]:
stability_df_LIME=stability_df_LIME.transpose()
stability_df_LIME

,0
Parkinson's,0.849006
Breast Cancer,0.877186
Adult Income,0.466304
Boston,0.796952
Body Fat,0.903673


UNRAVEL-L

In [14]:
def unravel_lime(model_path, x0, n_iterations=10):
    # Load the pre-trained model
    model = load(model_path)
    
    # Initialize LIME Explainer
    explainer = LimeTabularExplainer(
        training_data=X_train_parkinsons,  # Use the training data corresponding to the model
        feature_names=feature_names_parkinsons,  # Adjust based on the dataset being processed
        mode='classification' if 'svm' in model_path else 'regression'
    )

    # Initialize dataset D for storing explanations
    D = []
    explanations = []
    
    # Add the initial instance's explanation
    exp = explainer.explain_instance(x0, model.predict, num_features=5)
    explanations.append(exp.as_list())  # Store the explanation as a list
    D.append((x0, model.predict([x0])[0]))  # Initial data point with prediction

    for l in range(n_iterations):
        # Perturb the initial instance to create new samples
        new_samples = [x0 + np.random.normal(0, 0.1, size=x0.shape) for _ in range(10)]
        
        for sample in new_samples:
            # Get prediction for the new sample
            pred = model.predict([sample])[0]
            D.append((sample, pred))
            
            # Get LIME explanation for the new sample
            exp = explainer.explain_instance(sample, model.predict, num_features=5)
            explanations.append(exp.as_list())

    return D, explanations  # Return both surrogate data and explanations


In [15]:
datasets = {
    'parkinsons': {
        'model_path': 'models/parkinsons_svm_model.pkl',
        'initial_instance': X_test_parkinsons[0],  # Example instance from test set
        'iterations': 10,
        'X_train': X_train_parkinsons,  # Training data for Parkinson's dataset
    },
    'cancer': {
        'model_path': 'models/cancer_svm_model.pkl',
        'initial_instance': X_test_cancer[0],  # Example instance from test set
        'iterations': 10,
        'X_train': X_train_cancer,  # Training data for Cancer dataset
    },
    'adult': {
        'model_path': 'models/adult_svm_model.pkl',
        'initial_instance': X_test_adult[0],  # Example instance from test set
        'iterations': 10,
        'X_train': X_train_adult,  # Training data for Adult dataset
    },
    'boston': {
        'model_path': 'models/boston_extra_trees_model.pkl',
        'initial_instance': X_test_boston[0],  # Example instance from test set
        'iterations': 10,
        'X_train': X_train_boston,  # Training data for Boston dataset
    },
    'bodyfat': {
        'model_path': 'models/bodyfat_extra_trees_model.pkl',
        'initial_instance': X_test_bodyfat[0],  # Example instance from test set
        'iterations': 10,
        'X_train': X_train_bodyfat,  # Training data for BodyFat dataset
    },
}


In [16]:
# Initialize an empty dictionary to store models
models = {}


In [17]:

def train_svm_model(X_train, y_train):
    model = SVC(probability=True)  # Enable probability estimates
    model.fit(X_train, y_train)
    return model

In [18]:
# Train SVM models with probability estimates
models['parkinsons'] = train_svm_model(X_train_parkinsons, y_train_parkinsons)
models['cancer'] = train_svm_model(X_train_cancer, y_train_cancer)
models['adult'] = train_svm_model(X_train_adult, y_train_adult)

# Similarly, for Extra Trees models
from sklearn.ensemble import ExtraTreesRegressor

def train_extra_trees_model(X_train, y_train):
    model = ExtraTreesRegressor()
    model.fit(X_train, y_train)
    return model

models['boston'] = train_extra_trees_model(X_train_boston, y_train_boston)
models['bodyfat'] = train_extra_trees_model(X_train_bodyfat, y_train_bodyfat)


In [44]:
from lime.lime_tabular import LimeTabularExplainer

def unravel_lime(model_path, x0, X_train, n_iterations=10):
    # Load the model
    model = joblib.load(model_path)
    
    # Initialize LIME Explainer based on the model type
    if hasattr(model, "predict_proba"):  # Classification model
        explainer = LimeTabularExplainer(X_train, feature_names=list(range(X_train.shape[1])), mode='classification')
    else:  # Regression model
        explainer = LimeTabularExplainer(X_train, feature_names=list(range(X_train.shape[1])), mode='regression')

    explanations = []
    D = []

    for _ in range(n_iterations):
        # Add the initial instance's explanation
        exp = explainer.explain_instance(x0, model.predict, num_features=5)  # Use predict for regression
        explanations.append(exp.as_list())  # Store the explanation as a list
        D.append((x0, model.predict([x0])[0]))  # Initial data point with prediction

    return D, explanations


In [49]:
import joblib

In [50]:
stability_scores_lime = {
    'Dataset': [],
    'Stability Score': []
}



In [51]:
def calculate_jaccard_distance(X_i, X_j):
    # Convert explanations to sets of feature names (ignoring importance values)
    set_i = set([feature for feature, importance in X_i])
    set_j = set([feature for feature, importance in X_j])
    
    # Calculate intersection and union
    intersection = len(set_i & set_j)
    union = len(set_i | set_j)
    
    # Return Jaccard distance
    return intersection / union if union > 0 else 0


In [52]:
def calculate_stability_score(explanations):
    total_score = 0
    count = 0
    n = len(explanations)

    # Loop through all pairs of explanations
    for i in range(n):
        for j in range(i + 1, n):
            X_i = explanations[i]  # Each element is expected to be a list of tuples
            X_j = explanations[j]
            score = calculate_jaccard_distance(X_i, X_j)
            total_score += score
            count += 1

    # Return average score
    return total_score / count if count > 0 else 0


In [53]:
for dataset, params in tqdm(datasets.items()):
    D_final, explanations = unravel_lime(params['model_path'], params['initial_instance'], params['X_train'], n_iterations=params['iterations'])

    # Calculate stability using the updated function
    stability = calculate_stability_score(explanations)
    
    # Output the stability score
    print(f"Stability score for {dataset}: {stability}")


  0%|          | 0/5 [00:00<?, ?it/s]


NotImplementedError: LIME does not currently support classifier models without probability scores. If this conflicts with your use case, please let us know: https://github.com/datascienceinc/lime/issues/16

In [22]:

print(stability_df_lime)

NameError: name 'stability_df_lime' is not defined